In [ ]:
# pip install Dask ;
# pip install pysqlite ;
# pip install pandas ;
# pip install scikit-learn ;
# pip install group_lasso ;
# pip install numpy

# Mods

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from group_lasso import GroupLasso

import sqlite3
import warnings
warnings.filterwarnings('ignore')

# OLS

In [2]:
df = pd.read_csv("/home/r00323024/Xwines.csv")

In [3]:
dfnew = df.groupby(['WineID', 'Vintage','Type',"Grapes","Country","Acidity",'ABV','Body' ])['Rating'].mean().reset_index()

In [23]:
# unique_wines = df["WineName"].unique()
# file_path = "/home/r00323024/unique_wines.txt"
# with open(file_path, "w") as f:
#     for wine in unique_wines:
#         f.write(f"{wine}\n")

# Grapes = df["Grapes"].unique()
# file_path = "/home/r00323024/unique_Grapes.txt"
# with open(file_path, "w") as f:
#     for Grape in Grapes:
#         f.write(f"{Grape}\n")

dfnew

,WineID,Vintage,Type,Grapes,Country,Acidity,ABV,Body,Rating
0,100001,1988,Sparkling,['Muscat/Moscato'],Brazil,High,7.5,Medium-bodied,4.000000
1,100001,1999,Sparkling,['Muscat/Moscato'],Brazil,High,7.5,Medium-bodied,4.500000
2,100001,2007,Sparkling,['Muscat/Moscato'],Brazil,High,7.5,Medium-bodied,2.500000
3,100001,2008,Sparkling,['Muscat/Moscato'],Brazil,High,7.5,Medium-bodied,3.630952
4,100001,2009,Sparkling,['Muscat/Moscato'],Brazil,High,7.5,Medium-bodied,4.000000
...,...,...,...,...,...,...,...,...,...
247299,200591,2018,Rosé,['Modrý Portugal'],Czech Republic,Medium,12.5,Medium-bodied,3.000000
247300,200591,2020,Rosé,['Modrý Portugal'],Czech Republic,Medium,12.5,Medium-bodied,2.500000
247301,200709,1989,Dessert,['Tamaioasa Romaneasca'],Romania,High,11.5,Medium-bodied,3.500000
247302,200709,2008,Dessert,['Tamaioasa Romaneasca'],Romania,High,11.5,Medium-bodied,4.666667


In [4]:
%load_ext sql
%sql sqlite://

In [5]:
conn = sqlite3.connect(':memory:')  # Use ':memory:' for in-memory database

# df.to_sql('my_table', conn, index=False, if_exists='replace')
dfnew.to_sql('table_new', conn, index=False, if_exists='replace')


247304

In [6]:

Rating = pd.read_sql_query("SELECT Rating FROM table_new;", conn)
Type = pd.read_sql_query("SELECT Type FROM table_new;", conn)
Grapes = pd.read_sql_query("SELECT Grapes FROM table_new;", conn)
Country = pd.read_sql_query("SELECT Country FROM table_new;", conn)
Vintage = pd.read_sql_query("SELECT Vintage FROM table_new;", conn)
Acidity = pd.read_sql_query("SELECT Acidity FROM table_new;", conn)
ABV = pd.read_sql_query("SELECT ABV FROM table_new;", conn)
Body = pd.read_sql_query("SELECT Body FROM table_new;", conn)

In [7]:
Type_dum = pd.get_dummies(Type,drop_first=True, sparse=True).astype(int)
Grapes_dum = pd.get_dummies(Grapes,drop_first=True, sparse=True).astype(int)
Country_dum = pd.get_dummies(Country,drop_first=True, sparse=True).astype(int)
Vintage_dum = pd.get_dummies(Vintage,drop_first=True, sparse=True).astype(int)
Acidity_dum = pd.get_dummies(Acidity,drop_first=True, sparse=True).astype(int)
ABV_dum = pd.get_dummies(ABV,drop_first=True, sparse=True).astype(int)
Body_dum = pd.get_dummies(Body,drop_first=True, sparse=True).astype(int)

In [8]:
final_df = pd.DataFrame(index=Type_dum.index)

dfs = [Type_dum, Grapes_dum, Country_dum, Vintage_dum,Acidity_dum,Body_dum,ABV_dum]
for df in dfs:
    final_df = pd.concat([final_df, df], axis=1)

In [30]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247304 entries, 0 to 247303
Columns: 1656 entries, Type_Dessert/Port to ABV
dtypes: int64(1656)
memory usage: 3.1 GB


In [9]:
X = sm.add_constant(final_df)  # 添加截距项
y = Rating

model = sm.OLS(y, X)
results = model.fit()

In [11]:
file_path = "/home/r00323024/OLS.txt"
with open(file_path, "w") as f:
     f.write(results.summary().as_text())

# RF

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# # 假设df是你的DataFrame
# # 如果 "Body" 和 "Acidity" 是分类变量，需要先将其编码成数值型
# le = LabelEncoder()
# df['Body'] = le.fit_transform(df['Body'])
# df['Acidity'] = le.fit_transform(df['Acidity'])

# # 确保 Rating 是目标变量，并且是连续变量
# X = df[['Body', 'Acidity']]  # 注意此处的写法，应该是双重方括号
# y = df['Rating']

# # 划分训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 初始化随机森林回归器，可以指定 n_estimators 和其他参数
# reg = RandomForestRegressor(n_estimators=100)  # 也可以修改其他参数
# reg.fit(X_train, y_train)

# # 使用reg.predict(X_test)来进行预测


# Stat

In [ ]:
# plt.figure(figsize=(12, 36))  # 調整整體圖表的大小，高度增加以容納垂直排列的子圖

# plt.subplot(6, 1, 1)  # 6行1列的第1張圖
# sns.distplot(df2['ABV'])

# plt.subplot(6, 1, 2)  # 6行1列的第2張圖
# sns.histplot(df2['Body'], bins=len(df2['Body'].unique()))
# plt.xticks(rotation=45)

# plt.subplot(6, 1, 3)  # 6行1列的第3張圖
# sns.histplot(df2['Acidity'], bins=len(df2['Acidity'].unique()))

# plt.subplot(6, 1, 4)  # 6行1列的第4張圖
# sns.histplot(df2['Type'], bins=len(df2['Type'].unique()))

# plt.subplot(6, 1, 5)  # 6行1列的第5張圖
# sns.histplot(df2['Country'], bins=len(df2['Country'].unique()))
# plt.xticks(rotation=90)  # 旋轉標籤以避免重疊

# plt.subplot(6, 1, 6)  # 6行1列的第6張圖
# sns.histplot(df2['Elaborate'], bins=len(df2['Elaborate'].unique()))
# plt.xticks(rotation=45)

# plt.tight_layout()  # 調整子圖之間的間距
# plt.show()
